# Análise Global de Diversidade

In [71]:
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")

In [72]:
ANOS = [2015, 2016, 2017, 2018, 2019]

In [73]:
data_frame_diversidade = pd.read_csv('dataset/alunos_curso_2015_2019_tic_BA.csv')
data_frame_diversidade.head()

,NU_ANO_CENSO,CO_IES,TP_CATEGORIA_ADMINISTRATIVA,TP_ORGANIZACAO_ACADEMICA,CO_CURSO,CO_CURSO_POLO,TP_TURNO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,...,CO_PAIS_DESTINO,IN_MATRICULA,IN_CONCLUINTE,IN_INGRESSO_TOTAL,IN_INGRESSO_VAGA_NOVA,IN_INGRESSO_PROCESSO_SELETIVO,NU_ANO_INGRESSO,CO_UF,NO_CURSO,SG_IES
0,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA
1,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA
2,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA
3,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA
4,2015,578,1.0,1.0,13277.0,NaN,4.0,1.0,1.0,1.0,...,NaN,1.0,0.0,0.0,0.0,NaN,2013.0,29,CIÊNCIA DA COMPUTAÇÃO,UFBA


In [74]:
replace_map_cor_raca = {
    'TP_COR_RACA':{
        0: 'Não quis declarar',  
        1: 'Branca',
        2: 'Preta',
        3: 'Parda',
        4: 'Amarela',
        5: 'Indígena',
        9: 'Sem informação'
    }
}

In [75]:
replace_map_sexo = {
    'TP_SEXO':{
        1: 'Feminino',
        2: 'Masculino'
    }
}

In [76]:
data_frame_diversidade.replace(replace_map_cor_raca, inplace=True)

In [77]:
data_frame_diversidade.replace(replace_map_sexo, inplace=True)

In [78]:
data_frame_diversidade['TP_COR_RACA'].value_counts()

Não quis declarar    8399
Parda                5518
Preta                2654
Branca               1615
Amarela               107
Indígena               66
Sem informação         55
Name: TP_COR_RACA, dtype: int64

## Separação de dataframe por universidade

In [79]:
dataframe_IFBA = data_frame_diversidade[data_frame_diversidade['SG_IES'] == 'IFBA'] 
dataframe_UFBA = data_frame_diversidade[data_frame_diversidade['SG_IES'] == 'UFBA'] 
dataframe_UEFS = data_frame_diversidade[data_frame_diversidade['SG_IES'] == 'UEFS'] 
dataframe_UNEB = data_frame_diversidade[data_frame_diversidade['SG_IES'] == 'UNEB'] 
dataframe_IFBAIANO = data_frame_diversidade[data_frame_diversidade['SG_IES'] == 'IFBAIANO'] 
dataframe_UESB = data_frame_diversidade[data_frame_diversidade['SG_IES'] == 'UESB'] 
dataframe_UNIVASF= data_frame_diversidade[data_frame_diversidade['SG_IES'] == 'UNIVASF'] 
dataframe_UESC = data_frame_diversidade[data_frame_diversidade['SG_IES'] == 'UESC']

In [85]:
def calculo_desvio_padrao_colunas_categoricas(coluna, dataframe):
    return round(dataframe[coluna].std(), 4)

In [81]:
def min_max_scaler(dataframe, column_name):
    min_param = dataframe[column_name].min()
    max_param = dataframe[column_name].max()
    return (dataframe[column_name] - min_param) / (max_param- min_param)
        

In [94]:
lista_universidades = ['IFBA', 'UFBA', 'UEFS', 'UNEB', 'IFBAIANO', 'UESB', 'UNIVASF', 'UESC']

## Análise da dimensão Gênero

In [95]:
std_genero = {'IFBA': [], 'UFBA': [], 'UEFS': [], 'UNEB': [], 'IFBAIANO': [], 'UESB': [], 'UNIVASF': [], 'UESC': []}

In [96]:
dataframe_temp = data_frame_diversidade.groupby(['SG_IES', 'NU_ANO_CENSO'])
dataframe_temp = dataframe_temp['TP_SEXO'].value_counts().to_frame(name = 'TP_SEXO_QUANTIDADE').reset_index()
for ano in ANOS:
    current_dataframe = dataframe_temp[dataframe_temp['NU_ANO_CENSO'] == ano]
    current_dataframe['TP_SEXO_QUANTIDADE'] = min_max_scaler(current_dataframe, 'TP_SEXO_QUANTIDADE')
    for universidade in lista_universidades:        
        current_dataframe_temp = current_dataframe[current_dataframe['SG_IES'] == universidade] 
        std_result = calculo_desvio_padrao_colunas_categoricas('TP_SEXO_QUANTIDADE', current_dataframe_temp)    
        std_genero[universidade].append(std_result)
        

In [97]:
std_genero

{'IFBA': [0.3588, 0.3639, 0.4328, 0.4521, 0.495],
 'UFBA': [0.592, 0.5852, 0.5922, 0.5989, 0.6094],
 'UEFS': [0.276, 0.2404, 0.25, 0.2372, 0.2121],
 'UNEB': [0.1094, 0.1243, 0.1448, 0.1371, 0.107],
 'IFBAIANO': [0.1009, 0.1036, 0.0947, 0.0934, 0.109],
 'UESB': [0.1475, 0.1401, 0.1311, 0.1282, 0.1204],
 'UNIVASF': [0.1675, 0.1517, 0.1311, 0.1342, 0.1298],
 'UESC': [0.1485, 0.121, 0.1238, 0.1127, 0.0916]}

In [90]:
dataframe_temp[dataframe_temp['NU_ANO_CENSO'] == 2016]

,SG_IES,NU_ANO_CENSO,TP_SEXO,TP_SEXO_QUANTIDADE
2,IFBA,2016,Masculino,697
3,IFBA,2016,Feminino,258
12,IFBAIANO,2016,Masculino,187
13,IFBAIANO,2016,Feminino,62
22,UEFS,2016,Masculino,332
23,UEFS,2016,Feminino,42
32,UESB,2016,Masculino,196
33,UESB,2016,Feminino,27
42,UESC,2016,Masculino,165
43,UESC,2016,Feminino,19


In [88]:
dataframe_temp

,SG_IES,NU_ANO_CENSO,TP_SEXO,TP_SEXO_QUANTIDADE
0,IFBA,2015,Masculino,601
1,IFBA,2015,Feminino,224
2,IFBA,2016,Masculino,697
3,IFBA,2016,Feminino,258
4,IFBA,2017,Masculino,798
...,...,...,...,...
84,UNIVASF,2017,Feminino,24
85,UNIVASF,2018,Masculino,202
86,UNIVASF,2018,Feminino,21
87,UNIVASF,2019,Masculino,213


In [61]:
t = data_frame_diversidade.groupby(['SG_IES', 'NU_ANO_CENSO'])

In [62]:
t

In [63]:
dataframe = t['TP_SEXO'].value_counts().to_frame(name = 'count').reset_index()

In [64]:
r = dataframe[dataframe['NU_ANO_CENSO'] == 2015]

In [65]:
r['count'] = min_max_scaler(r, 'count')

In [68]:
d = r[r['SG_IES'] == 'IFBA']

In [70]:
d['count'].std()

0.3587876938187462